In [40]:
import os
import httplib2
import ConfigParser
import numbers
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from oauth2client.service_account import ServiceAccountCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [41]:
# This class allows to interact with GDrive
# To do it, a service account is require
# https://developers.google.com/identity/protocols/OAuth2ServiceAccount#creatinganaccount
class GDrive(object):
    def __init__(self, oauth_file):

        # Create OAuth credentials
        self._credentials = ServiceAccountCredentials.from_json_keyfile_name(oauth_file,
                                                                             ['https://www.googleapis.com/auth/drive'])
        self._credentials.authorize(httplib2.Http())
        self._gauth = GoogleAuth()
        self._gauth.credentials = self._credentials

    def version(self, resource_id, path, file_handle):
        # Creates object for interacting with GDrive API
        drive = GoogleDrive(self._gauth)

        # Gets remote file and update mime type if necessary
        file1 = drive.CreateFile({'id': resource_id})
        # Set mime type if different
        if file1['mimeType'] != file_handle:
            file1['mimeType'] = file_handle
        # Replaces contents with local file
        file1.SetContentFile(path)
        file1.Upload()

    def download(self, resource_id, file_name, path):
        # Creates object for interacting with GDrive API
        drive = GoogleDrive(self._gauth)

        # Downloads existing file
        file1 = drive.CreateFile({'id': resource_id})
        file1.GetContentFile(os.path.join(path, file_name))

        return file1

In [42]:
config = ConfigParser.RawConfigParser()
config.read("datasets.conf")

['datasets.conf']

In [43]:
# Downloads last MRR matrix
gdrive = GDrive(config.get('gdrive', 'auth_file'))
f = gdrive.download(config.get('gdrive', 'input_csv_file_id'),
                    config.get('mrr_input', 'file_name'),
                    config.get('mrr_input', 'file_dir'))

As we are accessing GDrive with a service account, remote file needs to be shared with it.

In [44]:
# Loads matrix to pandas and sets column names
input_file = os.path.join(config.get('mrr_input', 'file_dir'), config.get('mrr_input', 'file_name'))
customers_df = pd.read_csv(input_file, sep=',', header=0)

In [45]:
customers_df

,account_id,name,email,created_at,updated_at,current_plan,plan_ends_at,2015-08-01,2015-09-01,2015-10-01,2015-11-01,2015-12-01,2016-01-01,2016-02-01,2016-03-01,2016-04-01
0,1000,John,john@aaa.com,2015-08-01,2015-08-02,large,2015-08-02,100,100,100,100,100,100,100,100,100
1,1001,Sue,sam@aaa.com,2015-08-02,2015-08-03,large,2015-08-03,NaN,NaN,NaN,NaN,50,50,50,100,NaN
2,1002,Grace,george@aaa.com,2015-08-03,2015-08-04,small,2015-08-04,NaN,NaN,NaN,50,50,NaN,NaN,NaN,NaN
3,1003,Dan,dan@aaa.com,2015-08-04,2015-08-05,small,2015-08-05,NaN,NaN,NaN,NaN,NaN,100,100,50,NaN
4,1004,Mike,michelle@aaa.com,2015-08-05,2015-08-06,small,2015-08-06,NaN,NaN,NaN,NaN,50,50,50,50,NaN
5,1005,Lisa,lisa@aaa.com,2015-08-06,2015-08-07,large+,2015-08-07,NaN,NaN,NaN,NaN,75,75,75,150,150
6,1006,Grant,george@aaa.com,2015-08-07,2015-08-08,free,2015-08-08,NaN,NaN,75,75,75,NaN,NaN,NaN,NaN
7,1007,George,george@aaa.com,2015-08-08,2015-08-09,free,2015-08-09,NaN,NaN,NaN,NaN,100,100,100,NaN,NaN
8,1008,Betty,bernie@aaa.com,2015-08-09,2015-08-10,large+,2015-08-10,NaN,NaN,100,100,100,100,100,150,NaN
9,1009,Patty,bernie@aaa.com,2015-08-10,2015-08-11,small+,2015-08-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75,NaN


In [47]:
# Removes all invoice columns but last two completed periods (month-1, month-2)
today = datetime.now().date()
current_period = today.replace(day=1) - relativedelta(months=1)
prior_period = today.replace(day=1) - relativedelta(months=2)

# Removes non-used periods
columns_to_remove = list(customers_df.columns.values)
required_columns = ('account_id', 'name', 'email', 'created_at', 'current_plan',
                    'plan_ends_at', str(current_period), str(prior_period))
for c in required_columns:
    columns_to_remove.remove(c)

# Keeps only last two complete invoicing periods
customers_df.drop(columns_to_remove, axis=1, inplace=True)

# Rename invoicing period columns
customers_df.rename(columns={str(prior_period): 'prior_invoiced', 
                             str(current_period): 'current_invoiced'}, 
                    inplace=True)

In [48]:
# Cleans na values
customers_df.loc[:, 'prior_invoiced'].fillna(0, inplace=True)
customers_df.loc[:, 'current_invoiced'].fillna(0, inplace=True)

We complete the dataset with two new attributes:
- behaviour: new, churn, upgrade or downgrade
- invoiced variation: invoice variation from prior month to current

In [49]:
# Enrich data
def is_valid_invoice(value):
    return isinstance(value, numbers.Real) and value > 0.0
    
def get_customer_behaviour(invoices):
    # input: (prior_invoiced, current_invoiced)
    # output: behaviour, invoice variation
    prior = invoices[0] if is_valid_invoice(invoices[0]) else 0.0
    current = invoices[1] if is_valid_invoice(invoices[1]) else 0.0

    if prior == 0.0 and current > 0.0:
        return 'New', round(current, 2)
    if prior > 0.0 and current == 0.0:
        return 'Churn', round(prior * -1, 2)
    if prior < current:
        return 'Upgrade', round(current - prior, 2)
    elif prior > current:
        return 'Downgrade', round(current - prior, 2)
    return 'No change', 0.00

customers_df[['behaviour', 'invoiced_var']] = customers_df[['prior_invoiced', 'current_invoiced']] \
                                              .apply(lambda x: pd.Series(get_customer_behaviour(x)), axis=1)

In [50]:
customers_df

,account_id,name,email,created_at,current_plan,plan_ends_at,prior_invoiced,current_invoiced,behaviour,invoiced_var
0,1000,John,john@aaa.com,2015-08-01,large,2015-08-02,100,100,No change,0
1,1001,Sue,sam@aaa.com,2015-08-02,large,2015-08-03,50,50,No change,0
2,1002,Grace,george@aaa.com,2015-08-03,small,2015-08-04,0,0,No change,0
3,1003,Dan,dan@aaa.com,2015-08-04,small,2015-08-05,100,100,No change,0
4,1004,Mike,michelle@aaa.com,2015-08-05,small,2015-08-06,50,50,No change,0
5,1005,Lisa,lisa@aaa.com,2015-08-06,large+,2015-08-07,75,75,No change,0
6,1006,Grant,george@aaa.com,2015-08-07,free,2015-08-08,0,0,No change,0
7,1007,George,george@aaa.com,2015-08-08,free,2015-08-09,100,100,No change,0
8,1008,Betty,bernie@aaa.com,2015-08-09,large+,2015-08-10,100,100,No change,0
9,1009,Patty,bernie@aaa.com,2015-08-10,small+,2015-08-11,0,0,No change,0


In [51]:
# Export to csv
output_file = os.path.join(config.get('output', 'local_file_dir'),
                           config.get('output', 'local_file_name'))
customers_df.to_csv(output_file, encoding='utf-8', index=False)

This example requires the file already exists in GDrive so we uploaded just an empty file in advance via web UI and kept the file ID. 
The reason is, as we are accesing GDrive using a service account, if we create a new file with this account, it will be created in GDrive

In [52]:
# Updates new version of file to GDrive
gdrive = GDrive(config.get('gdrive', 'auth_file'))
gdrive.version(config.get('gdrive', 'output_csv_file_id'),
               output_file,
               'text/csv')

That's it. Now new data is available in GDrive and can be imported to CartoDB!